# Villa Emo

Villa Emo è una villa veneta realizzata nei pressi di Fanzolo, a Vedelago, in provincia di Treviso, dall'architetto Andrea Palladio. L'opera, costruita probabilmente a partire dal 1558, fu commissionata dalla famiglia Emo di Venezia, famiglia di cui è rimasta nelle disponibilità fino al 2004. Nel seguente progetto verrà illustrata una sua possibile implementazione mediante l'uso del framework di grafica computazionale Pyplasm. 

![villaemo](assets/1200px-Villa_Emo.jpg)

![villaemo](assets/01.png)

![villaemo](assets/02.png)

![villaemo](assets/03.png)

![villaemo](assets/04.png)

![villaemo](assets/05.png)

![villaemo](assets/06.png)

# Le stagioni

Oltre a definire Villa Emo, il progetto è stato "decorato" circondandolo con la flora (alberi e cespugli). Essa in particolare varia a seconda della stagione corrente. In particolare quello che si fa è verificare quale sia la stagione corrente e successivamente il fogliame utilizzato come texture varia. In particolare in primavera si avranno alberi più "floreali", d'estate alberi più verdi, in autunno alberi con foglie rosse e infine d'inverno alberi con foglie secche.

Di seguito la funzione per determinare le stagioni:

In [2]:
def get_season(now):
    Y = 2000
    seasons = [(3, (date(Y, 1, 1), date(Y, 3, 20))),
               (0, (date(Y, 3, 21), date(Y, 6, 20))),
               (1, (date(Y, 6, 21), date(Y, 9, 22))),
               (2, (date(Y, 9, 23), date(Y, 12, 20))),
               (3, (date(Y, 12, 21), date(Y, 12, 31)))]
    if isinstance(now, datetime):
        now = now.date()
    now = now.replace(year=Y)
    return next(season for season, (start, end) in seasons
                if start <= now <= end)

A questo punto per distribuire con maggiore probabilità le foglie della stagione corrente, è stato utilizzato numpy. In particolare con la funzione random.choice è possibile scegliere un elemento di un array, con una particolare probabilità.

In [4]:
import numpy
def pattern():
    probability = [.1, .1, .1, .1]
    probability[int(stagione)] = .7
    tipologia = numpy.random.choice(numpy.arange(0, 4), p=probability)
    return str(tipologia)+'.jpg'

# Inverno

![villaemo](assets/07.png)

# Primavera

![villaemo](assets/08.png)

# Estate

![villaemo](assets/09.png)

# Autunno

![villaemo](assets/10.png)

## Pianta

Per quanto riguarda la pianta, si è proceduto alla sua realizzazioneutilizzando un software open source di grafica vettoriale (Inkscape). Successivamente usando il tool svg2lines, gli svg sono stati trasformati in linee rappresentate da due coordinate spaziali. A questo punto è stato necessario trasformare l'array di linee in un grafo. A tale scopo è stata implementata la funzione lines_to_graph:

In [1]:
from pyplasm import *
def lines_to_graph(lines):
    """
    Trasforma un array di linee in un grafo

    :param lines:
    :return:
    """
    keys = {}
    nodes = []
    edges = []

    i = 0

    for line in lines:
        if not (line[0], line[1]) in keys.keys():
            keys[(line[0], line[1])] = i
            nodes.append((line[0], line[1]))
            i = i + 1

        if not (line[2], line[3]) in keys.keys():
            keys[(line[2], line[3])] = i
            nodes.append((line[2], line[3]))
            i = i + 1

        edges.append((keys[(line[0], line[1])], (keys[(line[2], line[3])])))

    return nodes, edges

Evaluating fenvs.py..
...fenvs.py imported in 0.006526 seconds


A questo punto si è passati a definire la funzione tramezzi centrali, che per l'appunto descrive i tramezzi centrali di villa emo. In questo contesto per evitare di usare la funzione DIFFERENCE al fine di creare l'ingresso della villa, sono state create due piante, in particolare centrale e centrale bucato e si è proceduto a combinarle una sopra l'altra con la funzione TOP. Sempre al fine di ottimizzare le performance si è cercato di evitare tutte le letture del file system, hardcodando nel codice le coordinate spaziali delle piante.

In [2]:
def tramezzi_centrale():
    """
    Restituisce i tramezzi centrali

    :return:
    """

    def graph_to_tramezzo(g, h):
        return MULTEXTRUDE(OFFSET([40, 40])(STRUCT(MKPOLS(lines_to_graph(g)))))(h)

    centrale = [[4694, 43, 4694, 2000],
                [4694, 2000, 6807, 2000],
                [6807, 2000, 6807, 43],
                [6807, 43, 4694, 43],
                [4694, 43, 4694, 43]]
    centrale_bucato = [[5262, 2000, 4694, 2000],
                       [4694, 2000, 4694, 43],
                       [4694, 43, 6807, 43],
                       [6807, 43, 6807, 2000],
                       [6807, 2000, 6164, 2000]]

    return TOP([
        TOP([
            graph_to_tramezzo(centrale, 400),
            graph_to_tramezzo(centrale_bucato, 700),
        ]),
        graph_to_tramezzo(centrale, 100),
    ])

Approccio simile al precedente è stato utilizzato per i tramezzi laterali:

In [3]:
def tramezzi_laterali():
    """
    Restituisce i tramezzi laterali

    :return:
    """

    def graph_to_tramezzo(g, h):
        return MULTEXTRUDE(OFFSET([40, 40])(STRUCT(MKPOLS(lines_to_graph(g)))))(h)

    tramezzi_laterali = [[4690, 537, 13, 537],
                         [7, 1290, 4687, 1290],
                         [4110, 537, 4110, 1293],
                         [3693, 537, 3693, 1293],
                         [2937, 537, 2937, 1290],
                         [2539, 537, 2539, 1293],
                         [831, 535, 831, 1293],
                         [7, 535, 7, 1803],
                         [11497, 535, 6835, 535],
                         [6841, 1293, 11498, 1293],
                         [7402, 535, 7402, 1293],
                         [8545, 535, 8545, 1293],
                         [9295, 535, 9295, 1293],
                         [9777, 535, 9777, 1293],
                         [10685, 535, 10685, 1293],
                         [11498, 535, 11498, 1752]]

    return graph_to_tramezzo(tramezzi_laterali, 600)

# Tetto

In Villa Emo si hanno due tipologie di tetti. I tetti laterali (un triangolo estruso) e il tetto centrale (una piramide). Di seguito viene descritto l'approccio usato per realizzare questi due oggetti:

## Tetto laterale

Come detto il tetto laterale di Villa Emo è assimilabile a un triangolo estruso, di seguito l'implementazione:

In [4]:
def tetto_triangolo(depth, width):
    """
    Crea un tetto triangolare

    :param width:
    :param depth:
    :return:
    """
    _width = float(width)
    _depth = float(depth)
    return ROTATE([1, 2])(PI / 2)(ROTATE([2, 3])(PI / 2)(
        MULTEXTRUDE(MKPOL([[[0, 0], [_width / 2, _width / 4], [_width, 0]], [[1, 2, 3]], None]))(_depth)))

## Tetto centrale

In [5]:
def tetto_piramidale(depth, width, height):
    """
    Crea un tetto a punta

    :param width:
    :param depth:
    :param height:
    :return:
    """
    _width = float(width)
    _scale = float(depth) / _width
    _height = float(height)
    return SCALE(2)(_scale)(ROTATE([1, 2])(PI / 4)(CONE([_width * (1 / SQRT(2)), _height])(4)))

In [6]:
def tetto_piccolo():
    return STRUCT([
        tegole()(ROTATE([1,2])(PI/2)(tetto_triangolo(1000, 1300))),
        T(2)(1000)(texture("fronte.jpg")(ROTATE([1, 2])(PI / 2)(tetto_triangolo(1, 1300)))),
    ])

## Colonne

Nella villa sono presenti due tipologie di colonne diverse. Quelle più piccole laterali e quella più grandi presenti nell'ingresso:

In [7]:
def gorgegous_column(dm, h):
    cylndr = COMP([JOIN, TRUNCONE([dm / 2, .8 * dm / 2, h])])(24)
    torus_bot = COMP([JOIN, TORUS([dm / 12, dm / 2])])([8, 27])
    torus_top = COMP([JOIN, TORUS([.8 * (dm / 12), .8 * (dm / 2)])])([8, 24])
    base = COMP([T([1, 2])([7 * dm / -12, 7 * dm / -23]), CUBOID])(
        [7 * dm / 6, 7 * dm / 6, dm / 6])
    base_top = COMP([T([1, 2])([7 * dm / -12, 7 * dm / -12]), CUBOID])(
        [7 * dm / 6, 7 * dm / 6, dm / 6])
    capital = SUM([
        COMP([JOIN, TRUNCONE([.8 * dm / 2, 1.2 * dm / 2, h / 8])])(4),
        COMP([R([1, 2])(PI / 4), JOIN, TRUNCONE([.8 * dm / 2, 1.2 * dm / 2, h / 8])])(4)
    ])
    return TOP([TOP([TOP([TOP([TOP([base, torus_bot]), cylndr]), torus_top]), capital]), base_top])

In [8]:
def column(width, height, width_decoration, height_decoration):
    _width = float(width)
    _height = float(height)
    _width_decoration = float(width_decoration)
    _height_decoration = float(height_decoration)
    def get_base():
        translation = -(_width_decoration - _width) / 2
        return T([1, 2])([translation, translation])(CUBOID([_width_decoration, _width_decoration, _height_decoration]))

    return STRUCT([T(3)(_height)(get_base()), CUBOID([_width, _width, _height]), T(3)(0)(get_base())])

## Foresta

La funzione albero permette di generare un albero. In particolare con il primo parametro è possibile indicare quale tipologia di albero generare. Gli alberi disponibili sono tre. Una volta definito l'albero, è stata definita una foresta. Una foresta viene creata generando randomicamente N alberi in uno spazio ben definito:

In [9]:
def albero(treeType, rTree, hTree, hTrunk):
    ambient = [0.0, 0.0, 0.0, 1.0]
    diffuse = [0.459, 0.27157, 0.0000, 1]
    specular = [0.0225, 0.0225, 0.0225, 1.0]
    emission = [0.0, 0.0, 0.0, 1.0]
    shininess = [12.8]
    woodMaterial = ambient + diffuse + specular + emission + shininess
    textureExtended = 'foliage2.jpg'
    textureNormal = 'foliage.jpg'

    if treeType == 3:
        textureExtended = 'flower.jpg'
        treeType = 2

    hFoliage = hTree - hTrunk
    if treeType == 1:
        foliage = MKPOL([[[0, 0, 0], [0, rTree, 0], [0, rTree * 1.5 / 4, hFoliage * 1 / 3],
                          [0, rTree * 3 / 4, hFoliage * 1 / 3], [0, rTree * 1 / 4, hFoliage * 2 / 3],
                          [0, rTree * 2 / 4, hFoliage * 2 / 3],
                          [0, 0, hFoliage], [0, 0, hFoliage * 2 / 3], [0, 0, hFoliage * 1 / 3]],
                         [[1, 2, 3, 9], [9, 4, 5, 8], [8, 6, 7]], 1])
        foliage = texture(textureExtended)(foliage)
        circle1 = texture(textureNormal)(CYLINDER([rTree * 1 / 2, 0])(10))
        circle2 = texture(textureNormal)(CYLINDER([rTree * 3 / 8, 0])(10))
        circle3 = texture(textureNormal)(CYLINDER([rTree * 2 / 8, 0])(10))
        foliage = STRUCT([foliage, circle1, T(3)(hFoliage * 1 / 3), circle2, T(3)(hFoliage * 1 / 3), circle3])

    elif treeType == 2:
        foliage = T(3)(rTree * 1 / 2)(R([1, 3])(math.pi / 2)(CYLINDER([rTree * 1 / 2, 0])(20)))
        foliage = S([2, 3])([2, hFoliage / rTree])(foliage)
        foliage = texture(textureExtended)(foliage)
        circle1 = texture(textureExtended)(CYLINDER([rTree * 1 / 2, 0])(10))
        circle2 = texture(textureExtended)(CYLINDER([rTree * 5 / 6, 0])(10))
        circle3 = texture(textureExtended)(CYLINDER([rTree * 1 / 2, 0])(10))
        foliage = STRUCT(
            [foliage, T(3)(hFoliage * 1 / 4), circle1, T(3)(hFoliage * 1 / 4), circle2, T(3)(hFoliage * 1 / 4),
             circle3])

    foliage = STRUCT([foliage, S(2)(-1), foliage])
    i = 0
    while i < 3.14:
        i += 3.14 / 4
        foliage = STRUCT([foliage, R([1, 2])(i), foliage])

    trunk = MATERIAL(woodMaterial)(CYLINDER([rTree / 16, hTrunk])(10))
    tree = STRUCT([trunk, T(3)(hTrunk), foliage])

    return tree

## Arco

In [10]:

def arch(lateral_width, height, deep):

    _lateral_width = float(lateral_width)
    _height = float(height)
    _deep = float(deep)

    def bottomArc(d):
        return BEZIER(S1)([[0, 0], [0, 2 * d / 3], [d, 2 * d / 3], [d, 0]])

    def topArc(d):
        return BEZIER(S1)([[0, 2 * d / 3], [d, 2 * d / 3]])

    def arc2D(d):
        return BEZIER(S2)([bottomArc(d), topArc(d)])

    def arc3D(d):
        def arc3D1(w):
            arco = arc2D(d)
            dominio = PROD([INTERVALS(1)(8), INTERVALS(1)(1)])
            ar = MAP(arco)(dominio)
            domin = PROD([ar, QUOTE([1.5])])
            return COMP([T(2)(w), R([2, 3])(PI / 2)])(domin)

        return arc3D1

    def Interarc(d1, d2):
        def Interarc1(w):
            return CUBOID([d1, w, 2 * d2 / 3])

        return Interarc1

    def Xarc(d1, d2):
        def Xarc1(w):
            return RIGHT([RIGHT([Interarc(d1, d2)(w), arc3D(d2)(w)]), Interarc(d1, d2)(w)])

        return Xarc1

    return Xarc(_lateral_width, _height)(_deep)

In [11]:
from view import *

./assets/tegole.jpg
./assets/tegole.jpg
Child 3
Child 3
./assets/steps.jpg
./assets/atrio.jpg
./assets/foliage2.jpg
./assets/foliage.jpg
./assets/foliage.jpg
./assets/foliage.jpg
./assets/flower.jpg
./assets/flower.jpg
./assets/flower.jpg
./assets/flower.jpg
./assets/flower.jpg
./assets/flower.jpg
./assets/flower.jpg
./assets/flower.jpg
./assets/foliage2.jpg
./assets/foliage.jpg
./assets/foliage.jpg
./assets/foliage.jpg
./assets/foliage2.jpg
./assets/foliage2.jpg
./assets/foliage2.jpg
./assets/foliage2.jpg
./assets/foliage2.jpg
./assets/foliage2.jpg
./assets/foliage2.jpg
./assets/foliage2.jpg
./assets/foliage2.jpg
./assets/foliage2.jpg
./assets/foliage2.jpg
./assets/foliage2.jpg
./assets/foliage2.jpg
./assets/foliage.jpg
./assets/foliage.jpg
./assets/foliage.jpg
./assets/foliage2.jpg
./assets/foliage2.jpg
./assets/foliage2.jpg
./assets/foliage2.jpg
./assets/flower.jpg
./assets/flower.jpg
./assets/flower.jpg
./assets/flower.jpg
./assets/flower.jpg
./assets/flower.jpg
./assets/flower.jpg

./assets/foliage2.jpg
./assets/foliage2.jpg
./assets/foliage2.jpg
./assets/foliage2.jpg
./assets/foliage2.jpg
./assets/foliage2.jpg
./assets/foliage2.jpg
./assets/foliage.jpg
./assets/foliage.jpg
./assets/foliage.jpg
./assets/foliage2.jpg
./assets/foliage2.jpg
./assets/foliage2.jpg
./assets/foliage2.jpg
./assets/foliage2.jpg
./assets/foliage.jpg
./assets/foliage.jpg
./assets/foliage.jpg
./assets/foliage2.jpg
./assets/foliage2.jpg
./assets/foliage2.jpg
./assets/foliage2.jpg
./assets/foliage2.jpg
./assets/foliage.jpg
./assets/foliage.jpg
./assets/foliage.jpg
./assets/foliage2.jpg
./assets/foliage2.jpg
./assets/foliage2.jpg
./assets/foliage2.jpg
./assets/flower.jpg
./assets/flower.jpg
./assets/flower.jpg
./assets/flower.jpg
./assets/flower.jpg
./assets/flower.jpg
./assets/flower.jpg
./assets/flower.jpg
./assets/foliage2.jpg
./assets/foliage2.jpg
./assets/foliage2.jpg
./assets/foliage2.jpg
./assets/flower.jpg
./assets/flower.jpg
./assets/flower.jpg
./assets/flower.jpg
./assets/foliage2.jpg

In [12]:
quit()